In [ ]:
import os
import openai
import json

In [ ]:
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai == None:
    print("OPENAI_API_KEY environment variable not set")
    openai_api_key = input("Enter your API key: ")
openai.api_key = openai_api_key

In [ ]:
model_list = openai.Model.list()
print([m["id"] for m in dict(model_list)["data"]])
standard_model = "text-davinci-003"

In [ ]:
test_response = openai.Completion.create(
    model=standard_model, 
    prompt="Question: \"Say OK if you understand.\"\nAnswer: \"", 
    max_tokens=3,
    stop=["\""])

In [ ]:
print(test_response["choices"][0]["text"])

In [ ]:
def ask_for_stackoverflow_answer(question_title, question_body):
    prompt = f"""QUESTION TITLE: {question_title}\nQUESTION BODY:{question_body}\n\nANSWER:"""
    print(prompt)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful StackOverflow user answering questions using the StackOverflow formatting rules:\n**bold**\n*italics*\n`code`"},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000
    )

    return response.choices[0].message.content

In [ ]:
def ask_for_stackoverflow_edit(question_title, question_body):
    prompt = f"""QUESTION_TITLE: {question_title}\nQUESTION_BODY: {question_body}"""
    print(prompt)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": "You are a helpful StackOverflow admin that edits StackOverflow questions. You are given a question with title (QUESTION_TITLE) and a text body (QUESTION_BODY). \
                You must edit the QUESTION_TITLE to be a question from the narrative perspective (called EDITED_TITLE). You must edit the QUESTION_BODY to fit the StackOverflow style (called EDITED_BODY). You can use the following formatting rules:\n**bold**\n*italics*\n`code`"},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000
    )

    if "EDITED_BODY:" in response.choices[0].message.content:
        edited_title = response.choices[0].message.content.split("EDITED_BODY:")[0].split("EDITED_TITLE:")[1]
        edited_body = response.choices[0].message.content.split("EDITED_BODY:")[1]
    else:
        print(response.choices[0].message.content)
        raise ValueError("Response does not contain EDITED_BODY")
    return edited_title, edited_body

In [ ]:
def ask_for_stackoverflow_answer_completion(question_title, question_body):
    prompt = f"""Question on StackOverflow titled \"{question_title}\":\n{question_body}\n\n""" + """A short and consice answer in typical StackOverflow manner, using the required formatting for code snippets for StackOverflow answers:\n"""
    print(prompt)
    response = openai.Completion.create(
        model=standard_model, 
        prompt=prompt,
        max_tokens=1000)

    return response["choices"][0]["text"]

In [ ]:
# answer = ask_for_stackoverflow_edit("I want to get the current date in Python","I have a Python script and I want to display today's date in YYYY-MM-DD format. I tried using datetime but it did not work... How to do it?")
# print(answer)

